# CTSE Lecture Notes Chatbot using LangChain + HuggingFace
This notebook demonstrates a simple chatbot that answers questions based on CTSE lecture notes using a Retrieval-Augmented Generation (RAG) approach.

In [1]:
!pip install langchain langchain-community langchain-huggingface chromadb transformers unstructured sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import pipeline

# Step 1: Load the Lecture Notes PDF
We'll use LangChain's `PyPDFLoader` to extract text from the CTSE lecture notes.

In [3]:
loader = PyPDFLoader("data/ctse_notes.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} pages")

Loaded 966 pages


# Step 2: Split Documents into Chunks
We use `RecursiveCharacterTextSplitter` to chunk the content into overlapping sections.

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(docs)}")

Total chunks created: 2018


# Step 3: Generate Embeddings
We'll use HuggingFace Sentence Transformers for vector representation.

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Store Chunks in Chroma Vector DB
We'll persist the chunks and embeddings to local vector storage.

In [6]:
persist_directory = "./ctse_db"
vectordb = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory=persist_directory)

# Step 5: Load a Local LLM (FLAN-T5)
We'll use HuggingFace's pipeline for open-source inference.

In [7]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    max_length=512,
    do_sample=False
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


# Step 6: Setup the QA Chain
Combining retriever and LLM using LangChain's RetrievalQA.

In [8]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Step 7: Ask Questions (Notebook-Friendly)
You can ask any question based on the CTSE lecture notes.  
Type `'exit'` to end the interaction.

In [ ]:
# Interactive question loop (Jupyter Notebook friendly)
def interactive_ctse_bot():
    while True:
        query = input("\nAsk a question (or type 'exit' to quit): ")
        if query.lower().strip() == "exit":
            print("Exiting the chatbot session.")
            break
        answer = qa_chain.invoke({"query": query})
        print("Answer:", answer["result"])

# Call this function to start Q&A
interactive_ctse_bot()


Ask a question (or type 'exit' to quit):  What are the primary benefits of using AWS Cloud?


Answer: Scalability - Easily add/ remove resources as required • Reliability - Backed by reliable AWS network with proven track record of uptime and performance • Cost-effectiveness - Pay only for what you use • Security - Wide range of security features and services to protect your data • Innovation - 200+ fully featured services for a wide range of technologies, industries, and use cases
